# 1. Ambiguous nucleotides - removing sequencing errors
- check if they are abundant in certain sequences (higher than 1% could be because of sequencing errors => not good to use)
- check if there is a big los if we delete sequences with 1 or more ambiguous nucleotides
- what if juste delete the ambiguous nucleotides in the sequences themselves but don't delete the entire sequence?\
If we are able to delete these without losing a lot of important data (distribution remains the same for example, not a big loss of indidvidual species), it would simplify the OHE and kmers a lot (because only ATGC instead of ATGCRYSWKMBDHVN)

In [ ]:
import pandas as pd

In [ ]:
def how_many_ambi(dataframe):
    df = pd.read_csv(dataframe)
    searchfor = ['R', 'Y', 'S', 'W', 'K', 'M', 'B', 'D', 'H', 'V', 'N']
    df['AMBI'] = df['SEQ'].str.count('|'.join(searchfor))
    df['AMBIPERC'] = df['AMBI'] / df['LENGTH']
    print('Amount of ambiguous nucleotides were added to dataframe in absolute numbers and percentages')
    return df


def too_much_ambi(dataframe, threshold=0.01):
    df = dataframe
    df = df[df.AMBIPERC < threshold]
    print('Sequences containing more than 1% ambiguous nucleotides were deleted from dataframe')
    return df


def drop_all_ambi(dataframe):
    df = dataframe
    df = df[df.AMBIPERC == 0.0]
    return df

# 2. Length - removing too long or too short ITS
Technical standpoint: Neural networks require input having the same size, this is why we e.g. extract some features with kmer (because they all end up having same size). For OHE, we need to padd the sequences up with an ambiguous nucleotide to all have the same length (this could be explained more).\
Biological standpoint: average length of ITS region is 550bp(?), ITS regions much shorter and much longer than that maybe do not hold any biological meaning\
This is why we are checking the distribution of the lengths of ITS, where to cut (min and max) and how much we lose if we cut at min and max

In [ ]:
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv("Taxonomy_99_final.csv")
print(f"Length of shortest sequence: {df['LENGTH'].min()}")
print(f"Length of longest sequence: {df['LENGTH'].max()}")
print(f"Average length of sequences: {df['LENGTH'].mean():0.2f}")
upper = df[df.LENGTH <= 650]
lowerupper = upper[upper.LENGTH >= 450]
print(f"The amount of sequences between 450 and 650 from the dataset are: {lowerupper.shape[0]}.")
print(f"This is {lowerupper.shape[0]/df.shape[0]*100:0.2f}% of the original dataset. ")

# Project settings for plots
plt.style.use('bmh')
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = 'UGent Panno Text'
plt.rcParams['font.monospace'] = 'UGent Panno Text'
plt.rcParams['font.size'] = 10
plt.rcParams['axes.labelsize'] = 10
plt.rcParams['axes.labelweight'] = 'bold'
plt.rcParams['axes.titlesize'] = 10
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.titlesize'] = 12
# For main: (UGent blue: color='#1E64C8')
# For accents: (UGent yellow: color='#FFD200', linestyle='dashed', linewidth=2)
# Set an aspect ratio
width, height = plt.figaspect(4)
fig = plt.figure(figsize=(width,height), dpi=400)

df.plot.hist(color='#1E64C8', bins=94, align='right')
plt.title('Distribution of the lengths of ITS sequences')
plt.xlabel('Length of ITS')
plt.xticks(range(0, 1501, 50), rotation='vertical')
plt.axvline(x=450, color='#FFD200', linestyle='dashed', linewidth=2)
plt.gca().get_xticklabels()[9].set_weight('bold')
plt.axvline(x=650, color='#FFD200', linestyle='dashed', linewidth=2)
plt.gca().get_xticklabels()[13].set_weight('bold')
# svg ensures
plt.savefig('LengthDistribution.svg')
plt.show()

Length of shortest sequence: 142\
Length of longest sequence: 1501\
Average length of sequences: 542.38\
The amount of sequences between 450 and 650 from the dataset are: 90211.\
This is 87.03% of the original dataset.\

Plotting the lengths of the ITS sequences in a histogram gives can be seen in the following figure. Lengths of 450 and 650 were highlighted as they were chosen to be the minimum and maximum.\
![LengthDistribution](https://user-images.githubusercontent.com/127412115/236612727-aa08bb36-477f-42a6-b0ea-2549046ae406.svg)



# 3. Representation - making the dataset more balanced (- Augmentation?)
To make the dataset more balanced:
- working with a minimum requirement (e.g. a species needs at least 5 representations of sequences)
- working with a maximum requirement (e.g. overrepresented species get randomly deleted some sequences to reduce their abundance)
- adding reverse complements to the underrepresented species without changing the overrepresented species\
note: overrepresented=higher than average amount of sequences per species, underrepresented=lower than average amount of sequences per species